In [1]:
import pandas as pd
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import s3fs

In [2]:
#df = pd.read_csv('../data/NYC-Airbnb-2023.csv', low_memory=False)
df = pd.read_csv('s3://mlops-course-project/data/NYC-Airbnb-2023.csv', low_memory=False)

In [14]:
df.count()

id                                42931
name                              42919
host_id                           42931
host_name                         42926
neighbourhood_group               42931
neighbourhood                     42931
latitude                          42931
longitude                         42931
room_type                         42931
price                             42931
minimum_nights                    42931
number_of_reviews                 42931
last_review                       32627
reviews_per_month                 32627
calculated_host_listings_count    42931
availability_365                  42931
number_of_reviews_ltm             42931
license                               1
dtype: int64

In [3]:
categorical = ['neighbourhood', 'room_type']
numerical = ['availability_365']
target = ['price']
df = df[categorical + numerical + target]
df[categorical] = df[categorical].astype(str)
df = df[df.price <= df.price.quantile(q=0.95)]

In [4]:
df_train, df_test = train_test_split(df, test_size=0.2)
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [5]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

y_train = df_train[target]

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

72.7985241983876

In [6]:
tests_dicts = df_test[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(tests_dicts)

y_test = df_test[target]
y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

71.64694852421573

In [12]:
tests_dicts = {'neighbourhood': 'West Village',
  'room_type': 'Private room',
  'availability_365': 0}
type(tests_dicts)

dict

In [11]:
X_test = dv.transform(tests_dicts)

In [13]:
type(X_test)

scipy.sparse._csr.csr_matrix

In [7]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

mean_squared_error(y_test, y_pred, squared=False)

73.74938600587691

In [8]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)